In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Add the src directory to the Python path

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.Template import Template
from src.utils import download_data
from src.mappings import ideology_mapping

In [2]:
template = Template(categories = ['sc'], file_path= '../datasets/test.csv')


Loaded template from ../datasets/test.csv


In [3]:
df = pd.read_csv('../datasets/Equinox2020.05.2023_MSP.csv', sep = '|')
polity_df = download_data("https://seshatdata.com/api/core/polities/")

Downloaded 839 rows


In [4]:
column_mappings = {}
for column in df.columns:
    column_mappings[column] = column.lower().replace('.', '_').replace('-', '_')
column_mappings['Date.From'] = 'year_from'
column_mappings['Date.To'] = 'year_to'
column_mappings['Variable'] = 'name'
df.rename(columns=column_mappings, inplace=True)

In [5]:
df['polity_id'] = df.apply(lambda x: int(polity_df.loc[polity_df['name'] == x['polity'], 'id'].values[0]) if x['polity'] in polity_df['name'].values else None, axis=1)
df = df[df.polity_id.notna()]
df.polity_id = df.polity_id.astype(int)

df['polity_new_name'] = df.apply(lambda x: polity_df.loc[polity_df['id'] == x['polity_id'], 'new_name'].values[0] if x['polity_id'] in polity_df['id'].values else None, axis=1)

In [6]:
df['is_disputed'] = False
df['is_uncertain'] = df.value_note.apply(lambda x: True if x == 'uncertain' else False)

In [7]:
variables = df.name.unique()

for variable in variables:
    var_df = df.loc[df['name'] == variable]
    variable = variable.replace(' ', '-').lower()
    if (variable in ideology_mapping['MSP'].keys()):
        print(variable)
        variable = variable.lower()
        var_df.name = variable
        var_df.rename(columns={'value_from': variable}, inplace=True)
        template.add_to_template(var_df, variable)

moral-concern-is-primary


/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.name = variable
/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.rename(columns={'value_from': variable}, inplace=True)


Added moral-concern-is-primary dataset to template
moralizing-enforcement-is-certain


/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.name = variable
/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.rename(columns={'value_from': variable}, inplace=True)


Added moralizing-enforcement-is-certain dataset to template
moralizing-norms-are-broad
Added moralizing-norms-are-broad dataset to template
moralizing-enforcement-is-targeted


...

Added moralizing-enforcement-in-afterlife dataset to template
moralizing-enforcement-in-this-life


/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.name = variable
/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.rename(columns={'value_from': variable}, inplace=True)


Added moralizing-enforcement-in-this-life dataset to template
moralizing-enforcement-is-agentic


/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.name = variable
/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.rename(columns={'value_from': variable}, inplace=True)


Added moralizing-enforcement-is-agentic dataset to template


In [8]:
template.save_dataset("../datasets/MSP_template.csv")

Saved template to ../datasets/MSP_template.csv
